# Testing Different Seasons and Leagues Used to Train an EPL Model 

When searching for data, it’s essential to understand how a model's performance changes with different training datasets. This helps us determine what data to prioritize for optimal results.

In this notebook, we examine how the choice of leagues in the training data affects model performance when evaluated exclusively on the English Premier League (EPL). To do this, we will:

0. [Establish a Baseline Model](#baseline-training-on-all-available-epl-data-only):
    - The model will predict second-half-of-season results using first-half season statistics as features.
    - It will be trained on all available EPL data*.

1. [Experiment with Data Cutoffs](#experiment-1-cutting-training-data-to-recent-seasons):
    - We will assess the impact of restricting the training data to more recent EPL seasons by removing older seasons.

2. [Incorporate Additional Leagues](#experiment-2-extending-training-data-to-other-top-leagues-all-seasons):
    - We will explore how training on multiple leagues affects model performance when applied to EPL predictions.

3. [Combine Both Strategies](#experiment-3-training-on-all-top-leagues-recent-seasons):
    - We will train the model using data from multiple leagues but only from recent seasons. This will help determine whether including more leagues can compensate for a lack of historical EPL data should we encounter this issue in our search.

4. [Remove EPL Data Completely](#experiment-4-training-on-non-epl-data-only):
    - Finally, as a bonus, we will train on non-EPL data and test on EPL data to understand how important it is to represent the leagues we are predicting for in our training set.

*The dataset we are using backdates to 2010 however there are additional seasons available from the source.

By conducting these experiments, we aim to identify the most effective training data composition for predicting EPL outcomes.

The modelling we use in this notebook (particularly the concept of using aggregated first-half-of-season stats to predict results in the second half) does not necessarily reflect how our future model(s) will work. This is meerly a method that sufficiently represents the likely complexity of our future model(s) whilst allowing us to use easily available back-dated data.

The data used in this notebook was sourced from [Football-Data.co.uk](https://Football-Data.co.uk).



## Experimentation Prep

In [1]:
import pandas as pd
import mlflow
from functools import partial

import sys
import os

In [2]:
sys.path.append(os.path.abspath(os.path.join("../")))
from src.preprocess import create_dataframe
from src.experimentation import run_experiment
from src.helper_functions import generate_random_string

In [3]:
raw_data = pd.read_csv("../data/raw_games.csv")

In [4]:
data = create_dataframe(raw_data)

2025-02-24 19:46:22.135 | INFO     | src.preprocess:create_dataframe:12 - Starting dataset creation
2025-02-24 19:46:22.139 | INFO     | src.preprocess:create_dataframe:112 - Home and away dataframes created succesfully
2025-02-24 19:46:22.143 | INFO     | src.preprocess:create_dataframe:118 - Home and away dataframes succesfully combined and grouped
2025-02-24 19:46:22.152 | INFO     | src.preprocess:create_dataframe:150 - Team stats succesfully merged onto game data
2025-02-24 19:46:22.153 | INFO     | src.preprocess:create_dataframe:155 - Duplicate columns successfully dropped
2025-02-24 19:46:22.157 | INFO     | src.preprocess:create_dataframe:169 - Dataset creation complete
2025-02-24 19:46:22.157 | INFO     | src.preprocess:create_dataframe:170 - Dataset length: 12783


In [5]:
def list_remove(lst: list, remove: list) -> list:
    return [x for x in lst if x not in remove]

In [6]:
non_features = [
    "season",
    "div",
    "date",
    "h_team",
    "a_team",
    "ftr",
    "b365h",
    "b365d",
    "b365a",
    "h_win",
    "bookies_prob",
]
FEATURE_NAMES = list_remove(data.columns, non_features)
ALL_SEASONS = list(data["season"].drop_duplicates())
TEST_SEASONS = ["23_24", "24_25"]
VAL_SEASONS = ["21_22", "22_23"]

In [7]:
columns = FEATURE_NAMES + ["bookies_prob", "div", "season", "h_win"]

data = data[columns]

train_full = data[~data["season"].isin(VAL_SEASONS + TEST_SEASONS)]
train_full = train_full.drop(columns=["bookies_prob"])

val = data[data["season"].isin(VAL_SEASONS) & (data["div"] == "E0")]
val = val.drop(columns=["div"])

In [8]:
seasons_13 = list_remove(ALL_SEASONS, ["10_11", "11_12", "12_13"])
seasons_17 = list_remove(seasons_13, ["13_14", "14_15", "15_16"])

In [9]:
training_experiment = partial(
    run_experiment,
    experiment_name="data-needs-experiment",
    val_data=val,
    hidden_units=None,
    learning_rate=0.001,
    num_epochs=10000,
    num_samples=1000,
    num_batches=1,
    league_tag="EPL",
    return_model=False,
)

In [10]:
def get_run_metrics(run_id: str) -> pd.DataFrame:
    metrics = mlflow.search_runs(filter_string=f"tags.run_id = '{run_id}'")[
        [
            "tags.run_description",
            "params.n_train",
            "params.num_train_seasons",
            "metrics.train_auc",
            "metrics.val_auc",
            "metrics.train_mse",
            "metrics.val_mse",
            "metrics.val_mse_diff",
            "metrics.val_auc_diff",
        ]
    ]

    return metrics

## Baseline: Training on All Available EPL Data Only

In [11]:
train_bl = train_full.copy()
train_bl = train_bl.loc[train_bl["div"] == "E0"]
train_bl = train_bl.drop(columns="div")

In [12]:
bl_run_id = generate_random_string()
print(f"Experiment run ID: {bl_run_id}")

training_experiment(
    run_id=bl_run_id,
    train_data=train_bl,
    run_description="training on 2010-2021 EPL data",
)

Experiment run ID: A7jYPk


Validation Output Sampling Progress: 100%|██████████| 1000/1000 [00:25<00:00, 38.84it/s]


🏃 View run wistful-turtle-265 at: http://localhost:5001/#/experiments/3/runs/323f496b4ea44eb3a7fc7880bbd69b54
🧪 View experiment at: http://localhost:5001/#/experiments/3


In [13]:
run_data_bl = get_run_metrics(bl_run_id)
run_data_bl

,tags.run_description,params.n_train,params.num_train_seasons,metrics.train_auc,metrics.val_auc,metrics.train_mse,metrics.val_mse,metrics.val_mse_diff,metrics.val_auc_diff
0,training on 2010-2021 EPL data,2067,11,0.711299,0.717342,0.215326,0.211627,0.006569,-0.025538


## Experiment 1: Cutting Training Data To Recent Seasons 

### Part A: 2013-2021

In [14]:
train_e1a = train_full.copy()
train_e1a = train_e1a.loc[
    (train_e1a["div"] == "E0") & (train_e1a["season"].isin(seasons_13))
]
train_e1a = train_e1a.drop(columns="div")

In [15]:
e1a_run_id = generate_random_string()
print(f"Experiment run ID: {e1a_run_id}")

training_experiment(
    run_id=e1a_run_id,
    train_data=train_e1a,
    run_description="training on 2013-2021 EPL data",
)

Experiment run ID: 97pRTI


Validation Output Sampling Progress: 100%|██████████| 1000/1000 [00:25<00:00, 38.95it/s]

🏃 View run rare-mule-884 at: http://localhost:5001/#/experiments/3/runs/9333709b20724b31950b505892165c8d
🧪 View experiment at: http://localhost:5001/#/experiments/3


In [16]:
run_data_e1a = get_run_metrics(e1a_run_id)
run_data_all = pd.concat([run_data_bl, run_data_e1a])
run_data_e1a

,tags.run_description,params.n_train,params.num_train_seasons,metrics.train_auc,metrics.val_auc,metrics.train_mse,metrics.val_mse,metrics.val_mse_diff,metrics.val_auc_diff
0,training on 2013-2021 EPL data,1505,8,0.710949,0.710746,0.215527,0.213505,0.008447,-0.032135


### Part B: 2017-2021

In [17]:
train_e1b = train_full.copy()
train_e1b = train_e1b.loc[
    (train_e1b["div"] == "E0") & (train_e1b["season"].isin(seasons_17))
]
train_e1b = train_e1b.drop(columns="div")

In [18]:
e1b_run_id = generate_random_string()
print(f"Experiment run ID: {e1b_run_id}")

training_experiment(
    run_id=e1b_run_id,
    train_data=train_e1b,
    run_description="training on 2017-2021 EPL data",
)

Experiment run ID: mubu15


Validation Output Sampling Progress: 100%|██████████| 1000/1000 [00:25<00:00, 38.93it/s]

🏃 View run useful-lynx-514 at: http://localhost:5001/#/experiments/3/runs/8da196d93e3d44b08e15a75ee7ede010
🧪 View experiment at: http://localhost:5001/#/experiments/3


In [19]:
run_data_e1b = get_run_metrics(e1b_run_id)
run_data_all = pd.concat([run_data_all, run_data_e1b])
run_data_e1b

,tags.run_description,params.n_train,params.num_train_seasons,metrics.train_auc,metrics.val_auc,metrics.train_mse,metrics.val_mse,metrics.val_mse_diff,metrics.val_auc_diff
0,training on 2017-2021 EPL data,945,5,0.724135,0.708824,0.211811,0.214416,0.009357,-0.034056


## Experiment 2: Extending Training Data To Other Top Leagues (All Seasons)

In [20]:
train_e2 = train_full.copy()
train_e2 = train_e2.drop(columns="div")

In [21]:
e2_run_id = generate_random_string()
print(f"Experiment run ID: {e2_run_id}")

training_experiment(
    run_id=e2_run_id,
    train_data=train_e2,
    run_description="training on 2010-2021 all leagues data",
)

Experiment run ID: STFdAt


Validation Output Sampling Progress: 100%|██████████| 1000/1000 [00:25<00:00, 38.78it/s]

🏃 View run selective-mole-507 at: http://localhost:5001/#/experiments/3/runs/c0abcfc1e15f468f990690ff0b58f6a4
🧪 View experiment at: http://localhost:5001/#/experiments/3


In [22]:
run_data_e2 = get_run_metrics(e2_run_id)
run_data_all = pd.concat([run_data_all, run_data_e2])
run_data_e2

,tags.run_description,params.n_train,params.num_train_seasons,metrics.train_auc,metrics.val_auc,metrics.train_mse,metrics.val_mse,metrics.val_mse_diff,metrics.val_auc_diff
0,training on 2010-2021 all leagues data,9840,11,0.699511,0.711578,0.217842,0.212731,0.007673,-0.031303


## Experiment 3: Training on All Top Leagues (Recent Seasons)

### Part A: 2013-2021

In [23]:
train_e3a = train_full.copy()
train_e3a = train_e3a.loc[train_e3a["season"].isin(seasons_13)]
train_e3a = train_e3a.drop(columns="div")

In [24]:
e3a_run_id = generate_random_string()
print(f"Experiment run ID: {e3a_run_id}")

training_experiment(
    run_id=e3a_run_id,
    train_data=train_e3a,
    run_description="training on 2013-2021 all leagues data",
)

Experiment run ID: pKg0Zb


Validation Output Sampling Progress: 100%|██████████| 1000/1000 [00:26<00:00, 37.29it/s]

🏃 View run awesome-wolf-550 at: http://localhost:5001/#/experiments/3/runs/a1b0706a01e84807878d2cf4dae9bc80
🧪 View experiment at: http://localhost:5001/#/experiments/3


In [25]:
run_data_e3a = get_run_metrics(e3a_run_id)
run_data_all = pd.concat([run_data_all, run_data_e3a])
run_data_e3a

,tags.run_description,params.n_train,params.num_train_seasons,metrics.train_auc,metrics.val_auc,metrics.train_mse,metrics.val_mse,metrics.val_mse_diff,metrics.val_auc_diff
0,training on 2013-2021 all leagues data,7127,8,0.707514,0.711836,0.215668,0.213126,0.008068,-0.031045


### Part B: 2017-2021

In [26]:
train_e3b = train_full.copy()
train_e3b = train_e3b.loc[train_e3b["season"].isin(seasons_17)]
train_e3b = train_e3b.drop(columns="div")

In [27]:
e3b_run_id = generate_random_string()
print(f"Experiment run ID: {e3b_run_id}")

training_experiment(
    run_id=e3b_run_id,
    train_data=train_e3b,
    run_description="training on 2017-2021 all leagues data",
)

Experiment run ID: AgkOC2


Validation Output Sampling Progress: 100%|██████████| 1000/1000 [00:26<00:00, 38.44it/s]

🏃 View run bustling-hen-867 at: http://localhost:5001/#/experiments/3/runs/b5603adba2824b34ad9d057118754d40
🧪 View experiment at: http://localhost:5001/#/experiments/3


In [28]:
run_data_e3b = get_run_metrics(e3b_run_id)
run_data_all = pd.concat([run_data_all, run_data_e3b])
run_data_e3b

,tags.run_description,params.n_train,params.num_train_seasons,metrics.train_auc,metrics.val_auc,metrics.train_mse,metrics.val_mse,metrics.val_mse_diff,metrics.val_auc_diff
0,training on 2017-2021 all leagues data,4442,5,0.717922,0.709083,0.212375,0.214785,0.009727,-0.033798


## Experiment 4: Training on Non-EPL Data Only

In [29]:
train_e4 = train_full.copy()
train_e4 = train_e4.loc[train_e4["div"] != "E0"]
train_e4 = train_e4.drop(columns="div")

In [30]:
e4_run_id = generate_random_string()
print(f"Experiment run ID: {e4_run_id}")

training_experiment(
    run_id=e4_run_id,
    train_data=train_e4,
    run_description="training on 2010-2021 EPL removed data",
)

Experiment run ID: HZzPTV


Validation Output Sampling Progress: 100%|██████████| 1000/1000 [00:27<00:00, 36.38it/s]

🏃 View run righteous-snail-252 at: http://localhost:5001/#/experiments/3/runs/84a207c980fe496da1f153fc333a87df
🧪 View experiment at: http://localhost:5001/#/experiments/3


In [31]:
run_data_e4 = get_run_metrics(e4_run_id)
run_data_all = pd.concat([run_data_all, run_data_e4])
run_data_e4

,tags.run_description,params.n_train,params.num_train_seasons,metrics.train_auc,metrics.val_auc,metrics.train_mse,metrics.val_mse,metrics.val_mse_diff,metrics.val_auc_diff
0,training on 2010-2021 EPL removed data,7773,11,0.696776,0.712581,0.218247,0.212735,0.007676,-0.030299


## Comparison and Conclusion

In [32]:
run_data_all.drop(
    columns=["metrics.train_auc", "metrics.train_mse"]
).sort_values("metrics.val_mse", ascending=True)

,tags.run_description,params.n_train,params.num_train_seasons,metrics.val_auc,metrics.val_mse,metrics.val_mse_diff,metrics.val_auc_diff
0,training on 2010-2021 EPL data,2067,11,0.717342,0.211627,0.006569,-0.025538
0,training on 2010-2021 all leagues data,9840,11,0.711578,0.212731,0.007673,-0.031303
0,training on 2010-2021 EPL removed data,7773,11,0.712581,0.212735,0.007676,-0.030299
0,training on 2013-2021 all leagues data,7127,8,0.711836,0.213126,0.008068,-0.031045
0,training on 2013-2021 EPL data,1505,8,0.710746,0.213505,0.008447,-0.032135
0,training on 2017-2021 EPL data,945,5,0.708824,0.214416,0.009357,-0.034056
0,training on 2017-2021 all leagues data,4442,5,0.709083,0.214785,0.009727,-0.033798


A few important points:

1. We are dealing with very small discrepancies in the evaluation metrics that, at face value, seem negligible. However, when we examine the `val_mse_diff` and `val_auc_diff` columns—which represent the disparity in MSE and AUC between our model and the bookies' odds—we see that these differences are not as insignificant as they first appear. For example, while a $0.001$ difference in MSE might seem inconsequential, if the total difference between our model’s MSE and that of the bookies' odds is only $0.02$, then suddenly that $0.001$ decrease becomes more meaningful and something we should strive to avoid. It is important therefore that we use the `_diff` columns for context when disucssing changes in metrics between runs.

2. Due to the model's feature selection, we are only utilising half of the available data points per season (i.e., games in the second half of the season). This means that any conclusions we attribute to insufficient data may not hold to the same extent (if at all) if we adopt an approach that allows us to use most or all of each season’s games.

3. Similarly, we cannot rule out that a different approach could lead to entirely different conclusions for other reasons. For example, if we were to find different features that (for whatever reason) carry more predictive signal in later seasons, a model that heavily relies on such features would likely benefit from a more recent training set. However, the goal of this notebook is to develop a rough understanding of how substituting earlier EPL seasons for more recent foreign seasons might affect model performance. This is a trade-off we may have to consider if we want a more comprehensive feature set.